Nome: Beatriz Lourenço

Nome: Gabriel Yamashita

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [270]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [271]:
import tweepy
import math
import os.path
import pandas as pd
import json
import numpy as np
import re 
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***@Reme72277305***

In [272]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [273]:
#Produto escolhido:
produto = "McDonalds"

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [274]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break
# shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [275]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [276]:
# Abrindo o Treinamento

data = pd.read_excel("McDonalds.xlsx",sheet_name = "Treinamento")
colunas = ["Treinamento", "relevancia"]
data = data.loc[:,colunas]

def cleanup(text):
    punctuation = "[!\-.:?;,'#-@]"
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed    

In [277]:
# Separando em relevantes e irrelevantes e limpando os caracteres irrelevantes

rel = " ".join(data[data["relevancia"] == 1].Treinamento)
irrel = " ".join(data[data["relevancia"] == 0].Treinamento)

rel = cleanup(rel)
irrel = cleanup(irrel)

rel = pd.Series(rel.split())
irrel = pd.Series(irrel.split())

In [278]:
# Valores relativos
rel_value = rel.value_counts(True)
irrel_value = irrel.value_counts(True)

___

In [279]:
#Laplace Smoothing

alpha = 1
V = 1e6

rel_value_number = rel.value_counts()
sum_rel_value_number = rel_value_number.sum()

irrel_value_number = irrel.value_counts()
sum_irrel_value_number = irrel_value_number.sum()

prob_p_rel = (rel_value_number + 1)/(sum_rel_value_number + alpha*V)
prob_p_irrel = (irrel_value_number + 1)/(sum_irrel_value_number + alpha*V)

prob_p_desc_rel = alpha / (sum_rel_value_number + alpha*V)
prob_p_desc_irrel = alpha / (sum_irrel_value_number + alpha*V)

prob_p_rel = prob_p_rel.sort_values(ascending=False)

___

In [280]:
# Abrindo o Teste

data = pd.read_excel("McDonalds.xlsx",sheet_name = "Teste")
colunas = ["Teste", "relevancia"]
data = data.loc[:,colunas]

In [281]:
# Função para calcular a chance de ser relevante e irrelevante

def classifica_relevante (tweet):
    w = cleanup(tweet).split()
    prob = np.log(prob_p_rel.reindex(w).fillna(prob_p_desc_rel)).sum()
    return prob
data["probabilidade relevante"]=data.Teste.apply(classifica_relevante)
data_rel = data["probabilidade relevante"]

def classifica_irrelevante (tweet):
    w = cleanup(tweet).split()
    prob = np.log(prob_p_irrel.reindex(w).fillna(prob_p_desc_irrel)).sum()
    return prob
data["probabilidade irrelevante"]=data.Teste.apply(classifica_irrelevante)
data_irrel = data["probabilidade irrelevante"]

In [282]:
data

,Teste,relevancia,probabilidade relevante,probabilidade irrelevante
0,rt @malevolazx: te assusta @mcdonalds_br https...,0,-126.496576,-109.960594
1,rt @btsnotice: [💛👌🏼] - @bts_twt \n\n▪hey hey h...,0,-276.807292,-233.955635
2,"como vocês conseguem comer mcdonalds 3, 4 veze...",0,-102.098967,-94.120708
3,burguer king é melhor que mcdonalds,1,-73.284816,-67.669711
4,se odiar é facil. quero ver parar de comer mcd...,0,-189.442637,-174.135301
5,"i'm at @mcdonalds_br in rio de janeiro, rj htt...",0,-192.469141,-176.754027
6,@davidamente @pobleminha1 @mcdonalds_br vergon...,0,-179.297590,-161.867976
7,@mcdonalds_br tá difícil... 3x que compro a p...,1,-442.087748,-411.844715
8,@mailitaaa @mcdonalds_br alaaaa mano acho q mu...,0,-104.231653,-98.682420
9,@mcdonalds_br queria saber pq tiraram uma carn...,1,-141.521745,-133.829508


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [283]:
'''
Porcentagem de verdadeiros positivos (mensagens relevantes e que são classificadas como relevantes)
Porcentagem de falsos positivos (mensagens irrelevantes e que são classificadas como relevantes)
Porcentagem de verdadeiros negativos (mensagens irrelevantes e que são classificadas como irrelevantes)
Porcentagem de falsos negativos (mensagens relevantes e que são classificadas como irrelevantes)
'''

'\nPorcentagem de verdadeiros positivos (mensagens relevantes e que são classificadas como relevantes)\nPorcentagem de falsos positivos (mensagens irrelevantes e que são classificadas como relevantes)\nPorcentagem de verdadeiros negativos (mensagens irrelevantes e que são classificadas como irrelevantes)\nPorcentagem de falsos negativos (mensagens relevantes e que são classificadas como irrelevantes)\n'

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**